# Model using Neural Network

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
import tensorflow.keras as keras

## Load The Data to DataFrame

In [2]:
df = pd.read_csv('cleaned_data.csv')

## Check the Columns' Data Type

In [3]:
df.columns

Index(['gender', 'age', 'workout_experience', 'workout_time', 'weight',
       'height', 'fitness_goal'],
      dtype='object')

In [4]:
from sklearn.utils import shuffle

shuffled_df = shuffle(df, random_state = 42)

## Split Data for Training and Validation


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(shuffled_df, test_size = 0.2, random_state=42)

## The training and validation set size

In [6]:
print('training : ',train_df.shape[0], " | validation : ", val_df.shape[0])

training :  723  | validation :  181


## Function Transform Dataframe into Tensor Dataset

In [7]:
def dataframe_transform(df, label):

    labels = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size = len(df))

    return ds

## Transform Training and Validation Dataframe into Tensor Dataset

In [8]:
train_ds = dataframe_transform(train_df, 'fitness_goal')
val_ds   = dataframe_transform(val_df, 'fitness_goal')

## Batching the Dataset

In [9]:
train_ds = train_ds.batch(20)
val_ds   = val_ds.batch(20)

## Function to Encode Categorical Column

In [10]:
def encode_categorical_feature(column, name, dataset):

    lookup_class = IntegerLookup
    lookup = lookup_class(output_mode = "binary")

    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    lookup.adapt(column_ds)
    encoded_column = lookup(column)

    return encoded_column

## Function to Encode Numerical Column

In [11]:
def encode_numerical_feature(column, name, dataset):

    normalizer = Normalization()
    column_ds = dataset.map(lambda x, y: x[name])
    column_ds = column_ds.map(lambda x: tf.expand_dims(x, -1))

    normalizer.adapt(column_ds)
    encoded_column = normalizer(column)

    return encoded_column

## Creating Tensor object from the column

In [12]:
gender = keras.Input(shape = (1,), name = 'gender', dtype = 'int64')
age    = keras.Input(shape = (1,), name = 'age')
workout_experience = keras.Input(shape = (1, ), name = 'workout_experience')
workout_time = keras.Input(shape = (1,), name = 'workout_time')
weight       = keras.Input(shape = (1,), name = 'weight')
height       = keras.Input(shape = (1,), name = 'height')

In [13]:
all_columns = [ gender, age, workout_experience, workout_time, height, weight ]

## Encode columns based on their data type

In [14]:
gender_encoded = encode_categorical_feature(gender, 'gender', train_ds)
age_encoded = encode_numerical_feature(age, 'age', train_ds)
workout_experience_encoded = encode_numerical_feature(workout_experience, 'workout_experience', train_ds)
workout_time_encoded = encode_numerical_feature(workout_time, 'workout_time', train_ds)
weight_encoded = encode_numerical_feature(weight, 'weight', train_ds)
height_encoded = encode_numerical_feature(height, 'height', train_ds)


In [15]:
all_features = keras.layers.concatenate([gender_encoded, age_encoded, workout_experience_encoded, workout_time_encoded, height_encoded, weight_encoded])

## Build the model architecture

In [16]:
x = keras.layers.Dense(64, activation = 'relu')(all_features)
x = keras.layers.Dense(16, activation = 'relu')(x)
x = keras.layers.Dropout(0.7)(x)

output = keras.layers.Dense(1, activation = 'sigmoid')(x)

In [17]:
model  = keras.Model(all_columns, output)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Train the model

In [83]:
from IPython.core.display import display, HTML

# Enable horizontal scrolling for output
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Generate a long output
long_output = "A" * 1000

model.fit(train_ds, epochs=300, validation_data = val_ds)

Epoch 1/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5571 - accuracy: 0.6625 - val_loss: 1.1864 - val_accuracy: 0.5028
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5681 - accuracy: 0.6611 - val_loss: 1.0949 - val_accuracy: 0.5414
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5543 - accuracy: 0.6473 - val_loss: 1.0702 - val_accuracy: 0.5580
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5573 - accuracy: 0.6667 - val_loss: 1.0419 - val_accuracy: 0.5580
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5497 - accuracy: 0.6722 - val_loss: 1.1935 - val_accuracy: 0.5193
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5685 - accuracy: 0.6598 - val_loss: 1.1166 - val_accuracy: 0.5525
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5676 - accuracy: 0.6515 - val_loss: 1.0631 - val_accuracy: 0.5414
Epoch 8/300
3

Epoch 59/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5495 - accuracy: 0.6611 - val_loss: 1.0342 - val_accuracy: 0.5525
Epoch 60/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5559 - accuracy: 0.6598 - val_loss: 1.0294 - val_accuracy: 0.5359
Epoch 61/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5441 - accuracy: 0.6639 - val_loss: 1.1062 - val_accuracy: 0.5470
Epoch 62/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5634 - accuracy: 0.6791 - val_loss: 1.1100 - val_accuracy: 0.5138
Epoch 63/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5517 - accuracy: 0.6667 - val_loss: 1.0830 - val_accuracy: 0.5580
Epoch 64/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5417 - accuracy: 0.6459 - val_loss: 1.0438 - val_accuracy: 0.5580
Epoch 65/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5638 - accuracy: 0.6694 - val_loss: 0.9593 - val_accuracy: 0.5304
Epoch 

37/37 [==============================] - 0s 3ms/step - loss: 0.5478 - accuracy: 0.6736 - val_loss: 1.1623 - val_accuracy: 0.5414
Epoch 117/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5773 - accuracy: 0.6501 - val_loss: 1.0912 - val_accuracy: 0.5359
Epoch 118/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5499 - accuracy: 0.6611 - val_loss: 1.1342 - val_accuracy: 0.5304
Epoch 119/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5680 - accuracy: 0.6528 - val_loss: 1.1495 - val_accuracy: 0.5470
Epoch 120/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5453 - accuracy: 0.6556 - val_loss: 1.0540 - val_accuracy: 0.5414
Epoch 121/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5474 - accuracy: 0.6542 - val_loss: 1.1170 - val_accuracy: 0.5580
Epoch 122/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5604 - accuracy: 0.6598 - val_loss: 1.0744 - val_accuracy: 0.5359
Epoch 123/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5566 - accuracy: 0.6680 - val_loss: 1.1343 - val_accuracy: 0.5414
Epoch 174/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5547 - accuracy: 0.6694 - val_loss: 1.1382 - val_accuracy: 0.5359
Epoch 175/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5519 - accuracy: 0.6625 - val_loss: 1.1739 - val_accuracy: 0.5470
Epoch 176/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5399 - accuracy: 0.6611 - val_loss: 1.1815 - val_accuracy: 0.5304
Epoch 177/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5570 - accuracy: 0.6584 - val_loss: 1.1325 - val_accuracy: 0.5414
Epoch 178/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5449 - accuracy: 0.6708 - val_loss: 1.1615 - val_accuracy: 0.5635
Epoch 179/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5709 - accuracy: 0.6639 - val_loss: 1.0119 - val_accuracy: 0.5580
Epoch 180/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5591 - accuracy: 0.6625 - val_loss: 1.1220 - val_accuracy: 0.5580
Epoch 231/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5604 - accuracy: 0.6598 - val_loss: 1.1605 - val_accuracy: 0.5470
Epoch 232/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5809 - accuracy: 0.6390 - val_loss: 1.0882 - val_accuracy: 0.5249
Epoch 233/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5685 - accuracy: 0.6570 - val_loss: 1.1646 - val_accuracy: 0.5691
Epoch 234/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5715 - accuracy: 0.6542 - val_loss: 1.0350 - val_accuracy: 0.5470
Epoch 235/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5858 - accuracy: 0.6639 - val_loss: 1.1721 - val_accuracy: 0.5470
Epoch 236/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5747 - accuracy: 0.6639 - val_loss: 1.0699 - val_accuracy: 0.5470
Epoch 237/300

37/37 [==============================] - 0s 3ms/step - loss: 0.5676 - accuracy: 0.6473 - val_loss: 1.1875 - val_accuracy: 0.5580
Epoch 288/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5913 - accuracy: 0.6515 - val_loss: 1.0768 - val_accuracy: 0.5580
Epoch 289/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5573 - accuracy: 0.6667 - val_loss: 1.0858 - val_accuracy: 0.5691
Epoch 290/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5657 - accuracy: 0.6515 - val_loss: 1.1976 - val_accuracy: 0.5470
Epoch 291/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5570 - accuracy: 0.6598 - val_loss: 1.2425 - val_accuracy: 0.5525
Epoch 292/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5846 - accuracy: 0.6418 - val_loss: 1.1241 - val_accuracy: 0.5249
Epoch 293/300
37/37 [==============================] - 0s 3ms/step - loss: 0.5639 - accuracy: 0.6542 - val_loss: 1.1281 - val_accuracy: 0.5304
Epoch 294/300

In [ ]:
df.info()

In [69]:
input_data = {
    'gender' : 1,
    'age'    :21,
    'workout_experience' : 0,
    'workout_time' : 60,
    'weight' : 65,
    'height' : 170,
}

In [70]:
input_dict = {}

In [71]:
for feature, value in input_data.items() :
    input_dict[feature] = tf.convert_to_tensor([value])

In [72]:
input_dict

{'gender': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([1])>,
 'age': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([21])>,
 'workout_experience': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([0])>,
 'workout_time': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([60])>,
 'weight': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([65])>,
 'height': <tf.Tensor: shape=(1,), dtype=int32, numpy=array([170])>}

In [76]:
predictions = model.predict(input_dict)

1/1 [==============================] - 0s 34ms/step


In [77]:
predictions

array([[0.5033811]], dtype=float32)

In [78]:
if predictions[0][0] >= 0.5 :
    print("You should do muscle up !")
else :
    print("You should do weight loss !")

You should do muscle up !


In [79]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'model_1_v1.pkl')

INFO:tensorflow:Assets written to: ram://5beea795-d635-421c-8e75-4da21a571b51/assets


['model_1_v1.pkl']